<a href="https://colab.research.google.com/github/sanjay-thiyagarajan/score-predictor/blob/main/CricPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget 'https://internalapp.nptel.ac.in/contest/reference_docs/ipl_csv2.zip'
!mkdir '/content/dataset'
!unzip '/content/ipl_csv2.zip' -d '/content/dataset'

In [38]:
import pandas as pd
import numpy as np
import os
import glob
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [ ]:
train_parent = []
DATASET_DIR = '/content/dataset/'
for file in os.listdir(DATASET_DIR):
  filename = os.path.join(DATASET_DIR, file)
  df = pd.read_csv(filename, error_bad_lines=False, header=0)
  train_parent.append(df)
train_parent = pd.concat(train_parent, axis=0, ignore_index=True)

# **Checking with limited features**

In [85]:
featured_df = train_parent[['striker', 'runs_off_bat', 'bowler']]
featured_df = featured_df.dropna()
people_encoder = LabelEncoder()
people = list(np.array(featured_df[['striker']]).flatten()) + list(np.array(featured_df[['bowler']]).flatten())
people = list(set(people))
people_encoder.fit(people)
featured_df[['striker']] = pd.DataFrame(np.array(people_encoder.transform(featured_df[['striker']]), dtype='int32'))
featured_df[['bowler']] = pd.DataFrame(np.array(people_encoder.transform(featured_df[['bowler']]), dtype='int32'))
featured_df = featured_df.dropna()
featured_df

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,striker,runs_off_bat,bowler
0,331.0,0.0,1.0
1,331.0,0.0,1.0
2,331.0,0.0,1.0
3,474.0,0.0,1.0
4,474.0,0.0,1.0
...,...,...,...
388703,551.0,1.0,25.0
388704,551.0,1.0,25.0
388705,551.0,1.0,25.0
388706,157.0,1.0,25.0


In [86]:
X_train = np.array(featured_df.drop(columns=['runs_off_bat']))
Y_train = np.array(featured_df[['runs_off_bat']])
regressor = LinearRegression()
regressor.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [88]:
sum(regressor.predict(X_train))

array([481306.99999996])

In [89]:
sum(Y_train)

array([481307.])